In [128]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from pandas import DataFrame
from datetime import datetime
import time
import csv

# selenium
browser = webdriver.Chrome()

browser.set_window_size(1000,800)
url = "https://www.bigkinds.or.kr/v2/news/search.do"

browser.get(url)
time.sleep(0.2)

# 카테고리 - 범죄 선택
browser.find_element(By.XPATH, '//*[@id="collapse-step-2-body"]/div/div[1]/ul[1]/li[4]/a').click()

browser.find_element(By.XPATH,'//*[@id="filterTab04"]/li[1]/span').click()
time.sleep(2)

def pop():
    # 맨 위 뉴스 팝업
    browser.find_element(By.XPATH, '//*[@id="news-results"]/div[1]/div/div[2]/a/div/strong/span').click()
    time.sleep(2)

def gettitle():
    next_news_title = browser.find_element(By.CSS_SELECTOR, \
    '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li.nextNewsItem > dd > a > span')
    print('newstitle:', next_news_title.text)
    return next_news_title.text

def getbody():
    # 날짜 저장
    news_date = browser.find_elements(By.XPATH, '//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/div[1]/ul/li[1]')
    for value in news_date:
        print('newsdate:',value.text)
    # 하나의 뉴스기사 기준 본문 저장
    news_bodies = browser.find_elements(By.XPATH, '//*[@id="news-detail-modal"]/div/div/div[1]/div/div[2]')
    news_body =''
    for i in news_bodies:
        news_body += i.text
    print('newsbody:',news_body)
    time.sleep(2)

newsId = 0

print('newsId:',newsId)
pop()
news_title = browser.find_element(By.CSS_SELECTOR, '#news-results > div:nth-child(1) > div > div.cont > a > div > strong > span')
print('newstitle:', news_title.text)
getbody()
# 반복
while True:
    newsId +=1
    time.sleep(2)
    print('newsId:',newsId)
    next_news_title = gettitle()
    if (newsId % 10 == 0): # 다음페이지로 이동해야하는 경우
        browser.find_element(By.CSS_SELECTOR, '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li.nextNewsItem > dd > a').send_keys(Keys.ENTER)
        time.sleep(2)
        pop()
        getbody()
    else:
        browser.find_element(By.CSS_SELECTOR, '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li.nextNewsItem > dd > a > span').click()
        time.sleep(2)
        getbody()


newsId: 0
newstitle: 검찰, '테라·루나 폭락 사태' 신현성 구속영장 청구
newsdate: 2022-11-30
newsbody: 신현성 전 차이코퍼레이션 총괄대표.
[아시아경제 최석진 법조전문기자] 테라·루나 폭락 사태를 수사 중인 검찰이 권도형 대표와 함께 테라폼랩스를 창립한 37)에 대해 29일 구속영장을 청구했다.

서울남부지검 금융증권범죄 합동수사단(단장 단성한)과 금융조사2부(부장검사 채희만)는 이날 특정경제범죄법상 사기 및 배임, 자본시장법 위반, 유사수신행위규제법 위반 등 혐의로 신 대표 등 8명에 대한 구속영장을 청구했다.

이날 구속영장이 청구된 8명 중 신 대표를 비롯한 4명은 테라·루나의 초기 투자자이고, 나머지 4명은 테라·루나 기술 개발 핵심 인력들인 것으로 알려졌다.

2010년 소셜커머스 티몬을 창립한 신 대표는 사업을 시작하기 전에 발행된 암호화폐 루나를 보유하고 있다가 가격이 폭등한 시기 매도해 1400억원의 시세차익을 챙긴 혐의를 받고 있다.

검찰은 루나를 비롯한 암호화폐에 증권성이 있다고 보고 신 대표에게 사기적 부정거래 등 자본시장법에 규정된 범죄 혐의를 적용했다.

신 대표에게는 이와 함께 차이코퍼레이션이 보유한 고객정보를 테라폼랩스 등 별도의 법인에 유출한 혐의(전자금융거래법 위반)도 적용됐다.

검찰은 지난 17일부터 신 대표를 세 차례 피의자 신분으로 소환해 강도 높게 조사했다.

테라·루나 사태 피해자들은 검찰 수사를 피해 해외에 머무르고 있는 권 대표를 고발할 때 신 대표도 함께 수사해달라고 요청한 바 있다.

하지만 이번 사태의 핵심 인물인 권 대표가 해외에 머무르며 검찰 수사에 응하지 않고 있는 상황에서 검찰이 어쩔 수 없이 우선 신 대표의 신병 확보에 나선 것이라는 분석도 나오고 있다.

2021년 4월 한국 법인을 해산하고 싱가포르로 출국한 권 대표는 5월 9일 20억 달러 상당의 테라를 매도했다. 권 대표는 검찰 수사에 응하겠다는 입장을 밝힌 뒤에도 귀국하지 않았다.

결국 검찰은 지난

newsId: 5
newstitle: 여친 이별 통보에…감금·폭행하고 분변까지 먹인 20대
newsdate: 2022-11-30
newsbody: "다시 만나자" 스토킹하고 머리카락도 잘라…檢, 징역 5년 구형
사진은 기사와 직접적인 관련이 없습니다. 연합뉴스
이별 통보한 여자친구를 감금·폭행하고 반려견의 분변을 먹이는 등 ‘엽기 행각’을 벌인 20대 남성에게 검찰이 실형을 구형했다.


검찰은 29일 인천지법 형사14부(류경진 부장판사) 심리로 열린 결심공판에서 중감금치상과 스토킹 범죄의 처벌 등에 관한 법률 위반 혐의로 재판에 넘겨진 20대 남성 A씨에게 징역 5년을 선고해 달라고 재판부에 요청했다.


A씨는 지난 4월 2일 인천의 한 오피스텔에서 당시 사귀던 여자친구 B씨를 5시간가량 감금한 뒤 폭행한 혐의로 기소됐다.


B씨로부터 “헤어지자”는 말을 들은 A씨는 B씨의 집을 찾아가 손발을 테이프로 묶고 심하게 폭행했다. 감금한 상태에서 B씨에게 반려견의 분변을 강제로 먹이거나 머리카락을 자르기도 했다.


A씨에게 폭행당한 B씨는 늑골이 부러지는 등 중상을 입어 병원에서 치료를 받았다.


하지만 여기에 그치지 않고 A씨는 범행 이후 지난 6월에는 헤어진 B씨에게 “다시 만나자”며 지속해서 연락해 스토킹한 혐의도 받는다. 그는 중감금치상 혐의로 재판을 받던 중 이달 초 스토킹 사건으로 추가 기소됨에 따라 재판부는 두 사건을 병합했다.


A씨의 선고 공판은 내년 1월 12일 인천지법 413호 법정에서 열릴 예정이다.
newsId: 6
newstitle: 檢, ‘테라·루나’ 공동창업자 신현성 구속영장 청구
newsdate: 2022-11-30
newsbody: 2018년 9월14일 제주국제컨벤션센터에서 열린 '업비트 개발자 컨퍼런스 2018'에서 티몬 의장 겸 테라 공동창업자 신현성이 발표를 하고 있다. 연합뉴스
한국산 가상화폐 ‘테라·루나’ 폭락 사태를 수사 중인 검찰이 테라폼랩스 공동창업자인 신현성씨에 대한 구속영장을 청구했다.

서울남부지검 금융

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=107.0.5304.108)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D4ACD3+2075859]
	Ordinal0 [0x00CDEE61+1633889]
	Ordinal0 [0x00BDB680+571008]
	Ordinal0 [0x00C0647E+746622]
	Ordinal0 [0x00C05B5A+744282]
	Ordinal0 [0x00C2731C+881436]
	Ordinal0 [0x00C015BF+726463]
	Ordinal0 [0x00C27534+881972]
	Ordinal0 [0x00C3B56A+963946]
	Ordinal0 [0x00C27136+880950]
	Ordinal0 [0x00BFFEFD+720637]
	Ordinal0 [0x00C00F3F+724799]
	GetHandleVerifier [0x00FFEED2+2769538]
	GetHandleVerifier [0x00FF0D95+2711877]
	GetHandleVerifier [0x00DDA03A+521194]
	GetHandleVerifier [0x00DD8DA0+516432]
	Ordinal0 [0x00CE682C+1665068]
	Ordinal0 [0x00CEB128+1683752]
	Ordinal0 [0x00CEB215+1683989]
	Ordinal0 [0x00CF6484+1729668]
	BaseThreadInitThunk [0x75EC6939+25]
	RtlGetFullPathName_UEx [0x77B08FD2+1218]
	RtlGetFullPathName_UEx [0x77B08F9D+1165]
